In [18]:
import requests
import json
import numpy as np
from IPython.display import Markdown, display
import pandas as pd
import re
import json
from datetime import datetime, timedelta

In [19]:
api_key='-Y83kwbfpZfHUQ_qtwp9dDFKZibNuaL879AAaANbf_E'

In [167]:
def get_resp_oai(input_text, model):
    url = "https://llm.api.ai8.io/query_llm"
    data = {
        # Specify the model that you want to use
        "model": model,
        "messages": [
                    {"role": "system", "content": "You act as a highly intelligent recommendation system. Your job is to analyse the user and movie data and rank the top 20 candidate movies to the user is most likely to watch next based on their historical data provided and the last movie the user has watched."},
                    {"role": "user", "content": input_text}
        ]
    }
    headers = {'Authorization': api_key}
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        response_data = json.loads(response.content)
        model_response = extract_message_oai(response_data)
        return model_response
    else:
        return {"statusCode": response.status_code, "body": response.content}

def extract_message_oai(response_data):
    message_content = response_data.get("choices", [])[0].get("message", {}).get("content", "")
    # format the extracted message as markdown
    markdown_content = "---\n\n" + message_content + "\n\n---"
    return markdown_content

In [31]:
# function to convert response string into an executable format: prompt 3, 5
def formatter(resp_str):
    return f"Can you transform this string: {resp_str} into an executable string by extracting the python code and keeping the rest of the information as comments? Please place comment symbols where nessesary and keep in mind that the given response string will be executed in a python code chunk using the function :'exec(given_response_string)."

In [28]:
# function to prompt the model for error resolution purposes given the error encountered and previous response as inputs: prompt 2,4
def prompt_error(prev_response, err):
    return f"Given that the code you provided: {prev_response} gives this error: {err}, help me resolve it by changing the code you provided where nessesary, your reposne should be in an executable code format"

In [21]:
movies = pd.read_csv('./movies.csv')
movies.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


In [148]:
top_50_best_movies = movies.sort_values(by=['votes'], ascending=False)
top_50_best_movies = top_50_best_movies.head(50).rename(columns={"name":"movie_name"}).to_dict(orient="index")
# top_50_best_movies = extract_movie_names(top_50_best_movies)
top_50_best_movies = top_50_best_movies.values()
# top_50_best_movies

Multi-Shot Chain of Thought Prompts

In [17]:
def step_1():
    return "Firstly, we need a Python function that generates a synthetic dataset of user interactions with movies? It should take a DataFrame containing movie data as input and optionally allow specifying the number of users. The generated dataset should include user IDs, movie names, ratings, the number of times each movie was watched, and the dates on which the movies were watched."

In [52]:
def step_2():
    return f"Given your revious response{msg_11}, Then, think about generating the user interaction dataset with the function you designed and merging it with a movies dataset based on the movie title, the movie dataset can be acessed though this path {'./movies.csv'} the goal is to combine the two datasets so that we have additional information about each movie, such as genre, release year, etc., associated with the user interactions. Please ensure that the merge operation is performed correctly based on the movie title."

In [44]:
def step_3():
    return f"Given your previous reponse{msg_22}, Afterwards as a third step, we need to create a function that retrieves all data associated with a particular user ID from a DataFrame containing user interactions with movies? The function should take a user ID as input and return a dictionary where each key is the movie name and the corresponding value is a dictionary containing all features associated with that movie for the specified user. The features should include rating, times watched, watched on date, and any other relevant information. Please ensure that the data is sorted by the watched on date in descending order."

In [89]:
def step_4():
    return f"Given tour previous responses in chronocological order: {msg_11}, {msg_22}, {msg_33}, tou should use the function to gether all user data and Then we need to create a function that splits a dictionary containing data into training and test sets based on a given train ratio? The function should take the data dictionary and the train ratio as input parameters. It should then split the data such that the training set contains the specified ratio of the data, and the test set contains the remaining data. Additionally, please calculate the index up to which the training data extends and return it along with the training and test sets."

In [35]:
prompt1 = step_1()
msg_1 = get_resp_oai(prompt1,"gpt-4-0125-preview")
prompt11 = formatter(msg_1)
msg_11 = get_resp_oai(prompt11,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_11))

<div style='color: #34568B;'>

---

```python
# To generate a synthetic dataset of user interactions with movies, we can use the Pandas library in Python. You'll first need to make sure you have pandas and numpy installed in your Python environment. You can install them using pip if you haven't done so yet:
# pip install pandas numpy

import pandas as pd
import numpy as np
from datetime import timedelta, datetime

def generate_user_interactions(movie_df, num_users=100, rating_range=(1, 5), max_watches_per_movie=10, date_range=("2020-01-01", "2023-01-01")):
    # Generate user IDs
    user_ids = np.arange(1, num_users + 1)
    
    # Initialize an empty list to store user interactions
    interactions = []
    
    # Define the date range
    start_date = np.datetime64(date_range[0], 'D')
    end_date = np.datetime64(date_range[1], 'D')
    days = (end_date - start_date).astype(int)
    
    # Generate synthetic data
    for user_id in user_ids:
        # Randomly select a subset of movies for each user
        user_movie_sample = movie_df.sample(n=np.random.randint(1, len(movie_df)), replace=False)
        
        for _, movie in user_movie_sample.iterrows():
            movie_name = movie['Movie Name']
            num_watches = np.random.randint(1, max_watches_per_movie + 1)
            
            for _ in range(num_watches):
                date_watched = start_date + np.random.randint(days)
                rating = round(np.random.uniform(*rating_range), 1)
                
                interactions.append({
                    'User ID': user_id,
                    'Movie Name': movie_name,
                    'Rating': rating,
                    'Date Watched': date_watched
                })
    
    # Convert the list of dicts into a DataFrame
    interactions_df = pd.DataFrame(interactions)
    
    # Calculate the number of times each movie was watched
    watches_count = interactions_df.groupby('Movie Name').size().reset_index(name='Watches')
    interactions_df = interactions_df.merge(watches_count, on='Movie Name', how='left')
    
    return interactions_df

# Ensure that your movie_df DataFrame contains at least one column named 'Movie Name' with the names of the movies before passing it to this function. This function creates synthetic user interaction data with specified boundaries, adding variability in user interactions for more realistic testing of recommendation systems or other analysis tasks.
# Example usage:
# Assuming you have a movie_df DataFrame with a column 'Movie Name'
# movie_df = pd.DataFrame({'Movie Name': ['Movie A', 'Movie B', ...]})
# interactions_df = generate_user_interactions(movie_df, num_users=100)
```

---

In [37]:
exec(msg_11[len("---\n\n```python\n"):len(msg_11)-len("\n```\n\n---")])

In [54]:
prompt2 = step_2()
msg_2 = get_resp_oai(prompt2,"gpt-4-0125-preview")
prompt22 = formatter(msg_2)
msg_22 = get_resp_oai(prompt22,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_22))

<div style='color: #34568B;'>

---

```python
# To merge the user interactions dataset generated using the `generate_user_interactions` function with additional information about each movie, such as genre and release year from a CSV file (`movies.csv`), you'll first need to load the `movies.csv` file into a DataFrame. Then, you can perform a merge operation based on the movie titles. Here's how you can do this:

import pandas as pd

# Load the movie details dataset
movies_df = pd.read_csv('./movies.csv')

# Assume movie_df is derived from the movies_df for generating user interactions
# Make sure to adjust according to the actual column name in movies.csv for movie titles.
movie_df = movies_df[['Movie Name']]

# Generate user interactions dataset as previously described
interactions_df = generate_user_interactions(movie_df, num_users=100)

# Merge the interactions dataset with the movies dataset to include additional movie details like genres, release year, etc.
# Ensure the merge operation is based on the 'Movie Name' or the corresponding column title used for movie titles in movies.csv.
# Adjust 'Movie Name' in the merge function according to the actual column names.
merged_df = interactions_df.merge(movies_df, on='Movie Name', how='left')

# Now merged_df contains user interactions along with additional movie details.
print(merged_df.head())

# Important Notes:
# 1. **Consistency in Movie Titles:** Before performing the merge, you might want to clean and ensure consistency in movie titles across both DataFrames to avoid mismatches during the merge operation.
# 2. **Column Names:** The example uses 'Movie Name' as the key for the merge. If your 'movies.csv' file uses a different column name for movie titles, you'll need to adjust the merge function accordingly.
# 3. **Data Cleaning:** It's always a good practice to clean and preprocess your data (e.g., handling missing values, standardizing formats) before merging to ensure the integrity of the merged dataset.
```

---

In [117]:
exec(msg_22[len("---\n\n```python\n"):len(msg_22)-len("\n```\n\n---")].replace("movies_df = pd.read_csv('./movies.csv')","movies_df = pd.read_csv('./movies.csv').rename(columns={'name':'Movie Name'})"))
     

   User ID               Movie Name  Rating Date Watched  Watches rating  \
0        1  The Flower of My Secret     4.0   2020-03-26      286      R   
1        1  The Flower of My Secret     2.5   2022-05-22      286      R   
2        1  The Flower of My Secret     4.0   2022-12-22      286      R   
3        1  The Flower of My Secret     4.3   2021-05-26      286      R   
4        1  The Flower of My Secret     1.7   2020-02-08      286      R   

    genre  year                       released  score    votes  \
0  Comedy  1995  March 8, 1996 (United States)    7.0  11000.0   
1  Comedy  1995  March 8, 1996 (United States)    7.0  11000.0   
2  Comedy  1995  March 8, 1996 (United States)    7.0  11000.0   
3  Comedy  1995  March 8, 1996 (United States)    7.0  11000.0   
4  Comedy  1995  March 8, 1996 (United States)    7.0  11000.0   

          director           writer            star country  budget  \
0  Pedro Almodóvar  Pedro Almodóvar  Marisa Paredes   Spain     NaN   
1  P

In [84]:
prompt3 = step_3()
msg_3 = get_resp_oai(prompt3,"gpt-4-0125-preview")
prompt33 = formatter(msg_3)
msg_33 = get_resp_oai(prompt33,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_33))

<div style='color: #34568B;'>

---

```python
# To create a function that meets your requirements, you'll need to follow these steps:
#
# 1. Filter the merged DataFrame `merged_df` for the specified user ID.
# 2. Sort the data based on the 'watched on date' in descending order.
# 3. Construct a dictionary where each key is the movie name, and the value is another dictionary containing all relevant movie features for this user.
#
# Let's assume your merged DataFrame `merged_df` contains columns like `User ID`, `Movie Name`, `Rating`, `Times Watched`, and `Watched On` (with dates). Here is how you can implement this function:

def retrieve_user_data(user_id, interactions_df):
    """
    Retrieves all movie interaction data for a specified user ID.
    
    Args:
    - user_id (int): The ID of the user.
    - interactions_df (pd.DataFrame): The DataFrame containing user interactions with movies.
    
    Returns:
    - dict: A dictionary where keys are movie names and values are dictionaries of movie features.
    """

    # Filter the DataFrame for the specified user ID
    user_data = interactions_df[interactions_df['User ID'] == user_id]
    
    # Sort the data by 'Watched On' in descending order
    user_data_sorted = user_data.sort_values(by='Watched On', ascending=False)
    
    # Initialize an empty dictionary to hold the movie data
    user_movies = {}
    
    # Loop through the DataFrame and populate the dictionary
    for index, row in user_data_sorted.iterrows():
        movie_name = row['Movie Name']
        # Assume all relevant columns you want to include are known
        movie_info = {
            'Rating': row['Rating'],
            'Times Watched': row['Times Watched'],
            'Watched On': row['Watched On'],
            # Add other relevant features here
        }
        
        # Add movie data to the user_movies dictionary
        user_movies[movie_name] = movie_info
        
    return user_movies

# Example usage
# user_id = 1
# user_movies_data = retrieve_user_data(user_id, merged_df)
# print(user_movies_data)

# This function assumes your DataFrame is named `merged_df` and includes a simplistic representation of your data schema. Make sure you adjust the column names accordingly, such as `Rating`, `Times Watched`, and `Watched On`, to match the exact column names in your `merged_df`.

```

---

In [136]:
exec(msg_33[len("---\n\n```python\n"):len(msg_33)-len("\n```\n\n---")].replace("Watched On", "Date Watched").replace("Times Watched", "Watches"))

In [92]:
prompt4 = step_4()
msg_4 = get_resp_oai(prompt4,"gpt-4-0125-preview")
prompt44 = formatter(msg_4)
msg_44 = get_resp_oai(prompt44,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_44))

<div style='color: #34568B;'>

---

Certainly! Here's how you can transform the provided string into an executable string with the Python code extracted and the rest of the information as comments:

```python
# Based on your request, I will use the information provided in the previous steps to create a function that splits the user movie data dictionary into training and test sets based on a given train ratio. Following this, I'll calculate the index up to which the training data extends. Here's how you can implement such a function in Python:

def split_data(data_dict, train_ratio):
    """
    Splits the user movies data dictionary into training and test sets based on the given train ratio.

    Args:
    - data_dict (dict): The dictionary containing user movies data with each key being a movie name and its value being another dictionary of movie features.
    - train_ratio (float): The ratio of data to be included in the training set.

    Returns:
    - tuple: A tuple containing the training set as the first element, the test set as the second element, and the index up until which the training data extends.
    """
    # Calculate the number of items to include in the training data
    total_items = len(data_dict)
    train_size = int(total_items * train_ratio)

    # Split the dictionary into training and test sets
    train_data = dict(list(data_dict.items())[:train_size])
    test_data = dict(list(data_dict.items())[train_size:])

    return train_data, test_data, train_size

# Example usage
# Assume data_dict is your user movie data dictionary
# Assume train_ratio is 0.8 for an 80-20 split
# data_dict = user_movies_data  # From your retrieval function
# train_ratio = 0.8

# train_data, test_data, train_size = split_data(data_dict, train_ratio)
# print(f"Training Data size: {len(train_data)}, Test Data size: {len(test_data)}")
# print("Training Data Extends Up To Index:", train_size)

# This function calculates the number of entries to include in the training data based on the specified ratio, then it splits the original data dictionary into training and test sets accordingly. The size of the training data is also returned, which corresponds to the index up to which the training data extends in the original dataset. Please ensure to replace `user_movies_data` with your actual dictionary containing user movie data before running the example usage.
```

This string can now be directly executed in a Python environment with the function definition and the instructional comments preserved.

---

In [127]:
exec(msg_44[178:len(msg_44)-146])

Zero-Shot Chain of Thought Prompts

In [114]:
def cot_0shotA():
    prompt = f"""
    Given all the previous prompts and responses in chronologicall order : [{prompt1} : {msg_11}, {prompt2} : {msg_22}, {prompt3} : {msg_33}, {prompt4} : {msg_44}]
    Retrieve User Data: by using one the functions defined in the earlier responses and considered the merged dataset called merged_df. This function should retrieve all data associated with a specific user ID from a dataset containing user interactions with movies. Ensure that the data is sorted by the watched on date in descending order.
    Split Data for Training and Testing: Next, use another function defined in the earlier prompts a. This function should split the user data dictionary into training and test sets based on a given train ratio. Return the training set, test set, and the split index.
    Define Variables: Once the data is split, assign the training set to a variable named 'train', the test set to 'test', and store the split index in a variable named 'i'. Set 'historical_interactions' as equal to 'train', and extract the most recently interacted movie from 'train' using the split index, assigning it to 'most_recent_interacted_movie'.
    """
    return prompt 

In [115]:
prompt5 = cot_0shotA()
msg_5 = get_resp_oai(prompt5,"gpt-4-0125-preview")
prompt55 = formatter(msg_5)
msg_55 = get_resp_oai(prompt55,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_55))

<div style='color: #34568B;'>

---

```python
# The steps provided outline a process involving data retrieval and manipulation using designed functions from previous responses. Below is the code that follows these instructions:
#
# 1. **Retrieve User Data:** Use the function defined earlier to retrieve data for a specific user ID from `merged_df`.
# 2. **Split Data for Training and Testing:** Use another previously defined function to split the data into training and test sets.
# 3. **Define Variables:** Assign variables as instructed.
#
# To execute these steps, ensure you have the `merged_df` DataFrame available as a result of merging user interactions and movies datasets, as well as the earlier defined `retrieve_user_data` and `split_data` functions.

# Assuming merged_df is already defined and loaded as per previous instructions

# Step 1: Retrieve User Data for a specific user ID
user_id_example = 1  # Use an example User ID, change this according to your dataset's IDs
user_movies_data = retrieve_user_data(user_id_example, merged_df)

# Step 2: Split Data into Training and Test sets
train_ratio_example = 0.8  # Define an example train ratio
train, test, i = split_data(user_movies_data, train_ratio_example)

# Step 3: Define Variables as described
historical_interactions = train
most_recent_interacted_movie = list(historical_interactions.items())[i-1]  # Assuming the dict is sorted by watched date

# Print to show that variables are set accordingly
print("Training Data:", len(train))
print("Test Data:", len(test))
print("Split Index:", i)
print("Most Recently Interacted Movie:", most_recent_interacted_movie)

# This code assumes:
# - The `merged_df` DataFrame is already created and contains user interactions merged with movie details.
# - The functions `retrieve_user_data(user_id, interactions_df)` and `split_data(data_dict, train_ratio)` are defined as prescribed in the previous responses.
# - Adjust `user_id_example` to match an actual user ID from your dataset.
# - The `train_ratio_example` is adjustable based on the desired training-testing split ratio.
#
# Ensure the code snippet is run in an environment where `merged_df` and the necessary functions are previously defined. Adjust the user ID and train ratio as needed.
```

---

In [137]:
exec(msg_55[len("---\n\n```python\n"):len(msg_55)-len("\n```\n\n---")])

Training Data: 5912
Test Data: 1479
Split Index: 5912
Most Recently Interacted Movie: ('The Blues Brothers', {'Rating': 2.4, 'Watches': 275, 'Date Watched': Timestamp('2020-04-12 00:00:00')})


In [165]:
def cot_0shotB():
    candidate_movies = top_50_best_movies
    sorted_interactions = sorted(historical_interactions.items(), key=lambda x: x[1]['Date Watched'])
    # Retrieve the 50 most recent interactions
    most_recent_50_interactions = dict(sorted_truncations[:50])
    prompt = F"""
    Initial Context:
    "I've watched the following movies in the past in order: {most_recent_50_interactions}."
    "Note that my most recently watched movie is {most_recent_interacted_movie}."
    Candidate Movies:
    "Now there are 50 candidate movies that I can watch next: {candidate_movies}."
    Task Description:
    "Please rank the top {len(test)} movies by measuring the possibilities that I would like to watch next most, according to my watching history."
    Instructions:
    "Please think step by step."
    "Please show me your ranking results with order numbers."
    "Split your output with line breaks."
    "You MUST rank the given candidate movies."
    "You can not generate movies that are not in the given candidate list."
    Output Format:
    "Please provide the list in a JSON format where the keys are the rankings and the values are the movie names."
    """
    return prompt

In [168]:
prompt6 = cot_0shotB()
msg_6 = get_resp_oai(prompt6,"gpt-4-0125-preview")
prompt66 = formatter(msg_6)
msg_66 = get_resp_oai(prompt66,"gpt-4-0125-preview")
display(Markdown("<div style='color: #34568B;'>\n\n" + msg_66))

<div style='color: #34568B;'>

---

Certainly! Here's how you can transform the provided text into an executable string with Python code and comments:

```python
# To determine the ranking of the candidate movies you might like to watch next, I'll analyze your watching history to identify preferences in genres, ratings, and directors, which seem to influence your choices heavily. Given that the historical data shows a mix of genres but with certain movies and directors appearing more than once with high ratings, I'd prioritize those factors in making recommendations. 
# 
# Without specific user interaction data on the candidates (e.g., whether action movies are preferred over drama, or if a higher score has more impact on choice than the director's popularity), recommendations are generalized based on apparent patterns in the provided viewing history and the attributes of the candidate movies.
# 
# 1. Analyzing the historical data, it seems high-rated movies and certain directors (like Christopher Nolan) are recurrently explored.
# 2. The mixture of genres in your history suggests a broad but discerning taste, favoring highly acclaimed movies regardless of their genre.
# 
# Assuming an algorithm that weighs:
# 
# - Similarity in genre to previously highly-rated movies.
# - Directors repeated in your history or known for critical acclaim.
# - High scores as an indicating factor of wide acceptance and potentially aligning with your tastes.
# 
# Here are the results formatted as requested:

movies_ranking = {
  "1": "The Dark Knight",
  "2": "Inception",
  "3": "The Shawshank Redemption",
  "4": "Pulp Fiction",
  "5": "The Prestige",
  "6": "Interstellar",
  "7": "The Lord of the Rings: The Return of the King",
  "8": "The Lord of the Rings: The Fellowship of the Ring",
  "9": "The Lord of the Rings: The Two Towers",
  "10": "Fight Club",
  "11": "The Matrix",
  "12": "Se7en",
  "13": "The Dark Knight Rises",
  "14": "Forrest Gump",
  "15": "Gladiator",
  "16": "Saving Private Ryan",
  "17": "Schindler's List",
  "18": "Memento",
  "19": "The Silence of the Lambs",
  "20": "The Departed"
  # Note: Only top 20 movies are considered executable based on user request. Remaining are excluded for brevity.
}

# It's important to note that this ranking is highly speculative and should be taken as a suggestion rather than a definitive forecast of your preferences. It leverages visible trends in your provided watch history and how they might intersect with the characteristics of the candidate movies.
```

The executable part is the creation of the dictionary `movies_ranking`, which stores the top 20 movie recommendations based on the user's presumed preferences. The rest of the text is commented out to provide context without interfering with the code execution.

---

Evaluation 

In [169]:
# Define function to calculate Recall@k
def recall_at_k(actual, predicted, k):
    actual_set = set(actual)
    predicted_set = set(predicted[:k])
    intersection = actual_set.intersection(predicted_set)
    recall = len(intersection) / len(actual_set)
    return recall

# Define function to calculate Discounted Cumulated Gain (DCG) at k
def dcg_at_k(actual, predicted, k):
    dcg = 0
    for i in range(min(k, len(predicted))):
        if predicted[i] in actual:
            dcg += 1 / np.log2(i + 2)  # Rank starts from 1, hence i + 2
    return dcg

# Define function to calculate Ideal Discounted Cumulated Gain (IDCG) at k
def idcg_at_k(actual, k):
    idcg = sum(1 / np.log2(i + 2) for i in range(min(k, len(actual))))
    return idcg

# Define function to calculate Normalized Discounted Cumulated Gain (NDCG) at k
def ndcg_at_k(actual, predicted, k):
    dcg = dcg_at_k(actual, predicted, k)
    idcg = idcg_at_k(actual, k)
    if idcg == 0:
        return 0
    else:
        return dcg / idcg

# Repeat experiments and calculate average metrics
def evaluate_metrics(actual, predicted, k_values=[1, 10, 20], num_experiments=1):
    recall_results = []
    ndcg_results = {k: [] for k in k_values}
    
    # for _ in range(num_experiments):
    recall_experiment = []
    ndcg_experiment = {k: [] for k in k_values}
    
    # for k in k_values:
    recall_experiment.append(recall_at_k(actual, predicted,20))
    for k_value in k_values:
        ndcg_experiment[k_value].append(ndcg_at_k(actual, predicted, k_value))
    
    recall_results.append(recall_experiment)
    for k in k_values:
        ndcg_results[k].append(ndcg_experiment[k])
    
    # Calculate average metrics
    avg_recall = np.mean(recall_results, axis=0)
    avg_ndcg = {k: np.mean(ndcg_results[k], axis=0) for k in k_values}
    
    return avg_recall, avg_ndcg

In [181]:
# helper functions
def extract_json_from_string(input_string):
    # Use regular expression to find the JSON part
    json_match = re.search(r'```json\n\{(?:.|\n)*?\}\n```', input_string)
    if json_match:
        json_string = json_match.group(0)
        return json_string
    else:
        return None

def string_to_dictionary(json_string):
    # Remove leading and trailing whitespace, as well as newlines
    json_string = json_string.strip()

    # Remove '```json' if present
    if json_string.startswith('```json'):
        json_string = json_string[len('```json'):]

    # Remove '```' if present
    if json_string.endswith('```'):
        json_string = json_string[:-len('```')]

    # Parse the JSON string into a dictionary
    try:
        dictionary = json.loads(json_string)
        return dictionary
    except json.JSONDecodeError:
        print("Error: Unable to decode JSON string.")
        return None
        
def extract_movie_names(data_dict):
    movie_names = [entry for entry in data_dict.keys()]
    return movie_names

In [183]:
# Example usage
# actual = ["Titanic", "Forrest Gump", "Gladiator", "Schindler's List", "Braveheart", "The Sixth Sense", "The Dark Knight"]
# predicted = ["The Shawshank Redemption", "The Godfather", "The Dark Knight", "Pulp Fiction", "Fight Club", "The Lord of the Rings: The Fellowship of the Ring", "The Matrix", "Goodfellas", "Inception", "Saving Private Ryan", "The Silence of the Lambs", "The Green Mile", "Se7en", "The Lion King", "The Sixth Sense", "American History X", "The Prestige", "The Departed", "Eternal Sunshine of the Spotless Mind", "A Beautiful Mind"]
actual = extract_movie_names(test)
predicted = extract_json_from_string(msg_6)
predicted = string_to_dictionary(predicted)
predicted = list(predicted.values())
avg_recall, avg_ndcg = evaluate_metrics(actual, predicted)

print("Average Recall@k:", avg_recall)
print("Average NDCG@k:")
for k, value in avg_ndcg.items():
    print(f"NDCG@{k}:", value)

Average Recall@k: [0.0020284]
Average NDCG@k:
NDCG@1: [0.]
NDCG@10: [0.18340981]
NDCG@20: [0.15070499]
